# **Robótica Industrial - Aula 05 - Cinemática Direta - Convenção de Denavit e Hartenberg** 
Prof. Lucas Vago Santana

*Obs: O objetivo do código é ser didático. Portanto, ele pode não possuir a implementação mais eficiente.*

# **Exemplo 01:** 
* Visualização de um robô planar de duas juntas (RR) criado a partir dos parâmetros DH;
* Observe a função t_DH que calcula a transformação de Denavit e Hartenberg;
* Dirija-se ao final do código para manipular as variáveis e visualizar a projeção do robô.

In [2]:
import plotly.graph_objects as go
import numpy as np

######################################################################################
############################ FUNÇÕES DE TRANSFORMAÇÃO ################################
######################################################################################
#Cria tranformação homogênea a partir dos parâmetros de Denavit-Hartenberg
def t_DH(theta, d, a, alpha):
  theta = np.radians(theta)  
  alpha = np.radians(alpha)  
  ct = np.cos(theta)
  st = np.sin(theta)
  ca = np.cos(alpha)
  sa = np.sin(alpha)
  
  Rz =  np.array((  (ct, -st, 0, 0), 
                    (st,  ct, 0, 0),
                    ( 0,   0, 1, 0),
                    ( 0,   0, 0, 1) ))
  
  Tz = np.array((   (1, 0, 0, 0), 
                    (0, 1, 0, 0),
                    (0, 0, 1, d),
                    (0, 0, 0, 1) ))

  Rx =  np.array((  (1,  0,   0, 0), 
                    (0, ca, -sa, 0),
                    (0, sa,  ca, 0),
                    (0,  0,   0, 1) ))
  
  Tx = np.array((   (1, 0, 0, a), 
                    (0, 1, 0, 0),
                    (0, 0, 1, 0),
                    (0, 0, 0, 1) ))
  
  return np.linalg.multi_dot([Rz, Tz, Tx, Rx])

######################################################################################
############################### PLOT DO ROBÔ #########################################
######################################################################################
def plot2jrobot(q1, a1, q2, a2):
  """
    Criação das coordenadas dos elos do robô, baseando-se nos parâmetros DH
    Parâmetros DH:
        theta   d   a   alpha
    1     q1    0   a1    0
    2     q2    0   a2    0
  """
  T_DH1 = t_DH(q1, 0, a1, 0) # Transformação homogênea do Elo 1
  T_DH2 = t_DH(q2, 0, a2, 0) # Transformação homogênea do Elo 2
  
  O1 = np.array((0,0,0,1)) # Origem no Elo 1

  L1 = np.linalg.multi_dot([T_DH1, O1]) # Coordenadas finais do Elo 1. Obtidas projetando O1 através da  duas transformações
  L2 = np.linalg.multi_dot([T_DH1, T_DH2, O1]) # Coordenadas finais do Elo 2. Obtidas projetando O1 através das duas transformações

  Vx = np.linalg.multi_dot([T_DH1, T_DH2, np.array((0.5,0,0,1))]) #Apenas plota o eixo x de coordenadas no efetuador final
  Vy = np.linalg.multi_dot([T_DH1, T_DH2, np.array((0,0.5,0,1))]) #Apenas plota o eixo y de coordenadas no efetuador final
  Vz = np.linalg.multi_dot([T_DH1, T_DH2, np.array((0,0,0.5,1))]) #Apenas plota o eixo z de coordenadas no efetuador final
  
  X1 = np.zeros((2, 3))
  X2 = np.zeros((2, 3))
  EX = np.zeros((2, 3))
  EY = np.zeros((2, 3))
  EZ = np.zeros((2, 3))

  X1[0,:] = O1[:3] #Pega os três primeiros elementos apenas
  X1[1,:] = L1[:3]

  X2[0,:] = L1[:3] #Pega os três primeiros elementos apenas. Origem Elo 2 é L1
  X2[1,:] = L2[:3]

  EX[0,:] = L2[:3]
  EX[1,:] = Vx[:3]

  EY[0,:] = L2[:3]
  EY[1,:] = Vy[:3]

  EZ[0,:] = L2[:3]
  EZ[1,:] = Vz[:3]

  fig = go.Figure()

  #Plot dos elos
  fig.add_trace(go.Scatter3d(x=X1[:,0], y=X1[:,1], z=X1[:,2], mode='markers+lines+text',text=["O1",""], name='L1', line=dict(color='blue',width=20)))
  fig.add_trace(go.Scatter3d(x=X2[:,0], y=X2[:,1], z=X2[:,2], mode='markers+lines+text', name='L2', line=dict(color='red',width=20)))
  
  # Plot dos eixos x, y e z no efetuador final
  fig.add_trace(go.Scatter3d(x=EX[:,0], y=EX[:,1], z=EX[:,2], mode='lines+text', text=["","x"], textfont=dict(color="darkblue"),name='Ex', line=dict(color='darkblue',width=5)))
  fig.add_trace(go.Scatter3d(x=EY[:,0], y=EY[:,1], z=EY[:,2], mode='lines+text', text=["","y"], textfont=dict(color="darkred"),name='Ey', line=dict(color='darkred',width=5)))
  fig.add_trace(go.Scatter3d(x=EZ[:,0], y=EZ[:,1], z=EZ[:,2], mode='lines+text', text=["","z"], textfont=dict(color="darkgreen"),name='Ez', line=dict(color='darkgreen',width=5)))

  #Configuração de layout da imagem gerada
  fig.update_layout(
      autosize=False,
      showlegend=False,
      width=400,
      height=400,
      margin=dict(l=0,r=0,t=0,b=0)
  )

  lim = 2.5
  fig.update_scenes(
      aspectmode= "cube",
      xaxis=dict(range=[-lim,lim]),
      yaxis=dict(range=[-lim,lim]),
      zaxis=dict(range=[-lim,lim])
  )

  print("Use o mouse para interagir com a figura gerada:")
  print("Posição do efetuador final: x = [%.1f] e y =[%.1f]" %  (L2[0], L2[1]))
  fig.show()

######################################################################################
############################ APLICANDO A CINEMÁTICA DIRETA ###########################
######################################################################################
[a1, a2] = [1, 2] #Tamanho das juntas
[q1, q2] = [0, 90] # Ângulo das juntas

plot2jrobot(q1, a1, q2, a2)

Use o mouse para interagir com a figura gerada:
Posição do efetuador final: x = [1.0] e y =[2.0]


# **Exemplo 02:** 
* Visualização de um robô 3D de três juntas (RRR) criado a partir dos parâmetros DH 

In [7]:
import plotly.graph_objects as go
import numpy as np

######################################################################################
############################ FUNÇÕES DE TRANSFORMAÇÃO ################################
######################################################################################
#Cria tranformação homogênea a partir dos parâmetros de Denavit-Hartenberg
def t_DH(theta, d, a, alpha):
  theta = np.radians(theta)  
  alpha = np.radians(alpha)  
  ct = np.cos(theta)
  st = np.sin(theta)
  ca = np.cos(alpha)
  sa = np.sin(alpha)
  
  Rz =  np.array((  (ct, -st, 0, 0), 
                    (st,  ct, 0, 0),
                    ( 0,   0, 1, 0),
                    ( 0,   0, 0, 1) ))
  
  Tz = np.array((   (1, 0, 0, 0), 
                    (0, 1, 0, 0),
                    (0, 0, 1, d),
                    (0, 0, 0, 1) ))

  Rx =  np.array((  (1,  0,   0, 0), 
                    (0, ca, -sa, 0),
                    (0, sa,  ca, 0),
                    (0,  0,   0, 1) ))
  
  Tx = np.array((   (1, 0, 0, a), 
                    (0, 1, 0, 0),
                    (0, 0, 1, 0),
                    (0, 0, 0, 1) ))
  
  return np.linalg.multi_dot([Rz, Tz, Tx, Rx])

######################################################################################
############################### PLOT DO ROBÔ #########################################
######################################################################################
def plot3jrobot(q1, a1, q2, a2, q3, a3):
  """
    Criação das coordenadas dos elos do robô, baseando-se nos parâmetros DH
    Parâmetros DH:
            theta       d     a     alpha
    1            q1     a1    0     90°
    2        90°+q2     0     a2    0°
    3       -90°+q3     0     a3    0°
  """
  T_DH1 = t_DH(q1, a1,  0, 90) # Transformação homogênea do Elo 1
  T_DH2 = t_DH(90+q2,  0, a2,  0) # Transformação homogênea do Elo 2
  T_DH3 = t_DH(-90+q3,  0, a3, 0) # Transformação homogênea do Elo 3
  
  O1 = np.array((0,0,0,1)) # Origem no Elo 1

  L1 = np.linalg.multi_dot([T_DH1, O1]) # Coordenadas finais do Elo 1. Obtidas projetando O1 através da  duas transformações
  L2 = np.linalg.multi_dot([T_DH1, T_DH2, O1]) # Coordenadas finais do Elo 2. Obtidas projetando O1 através das duas transformações
  L3 = np.linalg.multi_dot([T_DH1, T_DH2, T_DH3, O1]) # Coordenadas finais do Elo 2. Obtidas projetando O1 através das duas transformações

  X1 = np.zeros((2, 3))
  X2 = np.zeros((2, 3))
  X3 = np.zeros((2, 3))
  
  X1[0,:] = O1[:3] #Pega os três primeiros elementos apenas
  X1[1,:] = L1[:3]
  
  X2[0,:] = L1[:3] #Pega os três primeiros elementos apenas. Origem Elo 2 é L1
  X2[1,:] = L2[:3]

  X3[0,:] = L2[:3] #Pega os três primeiros elementos apenas. Origem Elo 2 é L1
  X3[1,:] = L3[:3]

  Vx1 = np.array((0.5,0,0,1))
  Vy1 = np.array((0,0.5,0,1))#np.linalg.multi_dot([T_DH1, T_DH2, T_DH3, np.array((0,0.5,0,1))]) #Apenas plota o eixo y de coordenadas no efetuador final
  Vz1 = np.array((0,0,0.5,1))#np.linalg.multi_dot([T_DH1, T_DH2, T_DH3, np.array((0,0,0.5,1))]) #Apenas plota o eixo z de coordenadas no efetuador final
  
  Vx2 = np.linalg.multi_dot([T_DH1, Vx1]) #Plota o eixo x de coordenadas no link 1
  Vy2 = np.linalg.multi_dot([T_DH1, Vy1]) #Plota o eixo y de coordenadas no link 1
  Vz2 = np.linalg.multi_dot([T_DH1, Vz1]) #Plota o eixo z de coordenadas no link 1

  Vx3 = np.linalg.multi_dot([T_DH1, T_DH2, Vx1]) #Plota o eixo x de coordenadas no link 2
  Vy3 = np.linalg.multi_dot([T_DH1, T_DH2, Vy1]) #Plota o eixo y de coordenadas no link 2
  Vz3 = np.linalg.multi_dot([T_DH1, T_DH2, Vz1]) #Plota o eixo z de coordenadas no link 2

  Vxe = np.linalg.multi_dot([T_DH1, T_DH2, T_DH3, Vx1]) #Plota o eixo x de coordenadas no link 3
  Vye = np.linalg.multi_dot([T_DH1, T_DH2, T_DH3, Vy1]) #Plota o eixo y de coordenadas no link 3
  Vze = np.linalg.multi_dot([T_DH1, T_DH2, T_DH3, Vz1]) #Plota o eixo z de coordenadas no link 3

  EX1 = np.zeros((2, 3))
  EY1 = np.zeros((2, 3))
  EZ1 = np.zeros((2, 3))

  EX2 = np.zeros((2, 3))
  EY2 = np.zeros((2, 3))
  EZ2 = np.zeros((2, 3))

  EX3 = np.zeros((2, 3))
  EY3 = np.zeros((2, 3))
  EZ3 = np.zeros((2, 3))
  
  EXe = np.zeros((2, 3))
  EYe = np.zeros((2, 3))
  EZe = np.zeros((2, 3))

  EX1[0,:] = O1[:3]
  EX1[1,:] = Vx1[:3] 
  EY1[0,:] = O1[:3]
  EY1[1,:] = Vy1[:3]
  EZ1[0,:] = O1[:3]
  EZ1[1,:] = Vz1[:3]

  EX2[0,:] = L1[:3]
  EX2[1,:] = Vx2[:3]
  EY2[0,:] = L1[:3]
  EY2[1,:] = Vy2[:3]
  EZ2[0,:] = L1[:3]
  EZ2[1,:] = Vz2[:3]

  EX3[0,:] = L2[:3]
  EX3[1,:] = Vx3[:3]
  EY3[0,:] = L2[:3]
  EY3[1,:] = Vy3[:3]
  EZ3[0,:] = L2[:3]
  EZ3[1,:] = Vz3[:3]

  EXe[0,:] = L3[:3]
  EXe[1,:] = Vxe[:3]
  EYe[0,:] = L3[:3]
  EYe[1,:] = Vye[:3]
  EZe[0,:] = L3[:3]
  EZe[1,:] = Vze[:3]

  fig = go.Figure()

  #Plot dos elos
  fig.add_trace(go.Scatter3d(x=X1[:,0], y=X1[:,1], z=X1[:,2], mode='markers+lines+text',text=["",""], name='L1', line=dict(color='blue',width=20)))
  fig.add_trace(go.Scatter3d(x=X2[:,0], y=X2[:,1], z=X2[:,2], mode='markers+lines+text', name='L2', line=dict(color='red',width=20)))
  fig.add_trace(go.Scatter3d(x=X3[:,0], y=X3[:,1], z=X3[:,2], mode='markers+lines+text', name='L2', line=dict(color='green',width=20)))

  # Plot dos eixos x, y e z no início do elo 1 (origem)
  fig.add_trace(go.Scatter3d(x=EX1[:,0], y=EX1[:,1], z=EX1[:,2], mode='lines+text', text=["","x1"], textfont=dict(color="darkblue"),name='Ex1', line=dict(color='darkblue',width=5)))
  fig.add_trace(go.Scatter3d(x=EY1[:,0], y=EY1[:,1], z=EY1[:,2], mode='lines+text', text=["","y1"], textfont=dict(color="darkred"),name='Ey1', line=dict(color='darkred',width=5)))
  fig.add_trace(go.Scatter3d(x=EZ1[:,0], y=EZ1[:,1], z=EZ1[:,2], mode='lines+text', text=["","z1"], textfont=dict(color="darkgreen"),name='Ez1', line=dict(color='darkgreen',width=5)))
  
  # Plot dos eixos x, y e z no início do elo 2
  fig.add_trace(go.Scatter3d(x=EX2[:,0], y=EX2[:,1], z=EX2[:,2], mode='lines+text', text=["","x2"], textfont=dict(color="darkblue"),name='Ex2', line=dict(color='darkblue',width=5)))
  fig.add_trace(go.Scatter3d(x=EY2[:,0], y=EY2[:,1], z=EY2[:,2], mode='lines+text', text=["","y2"], textfont=dict(color="darkred"),name='Ey2', line=dict(color='darkred',width=5)))
  fig.add_trace(go.Scatter3d(x=EZ2[:,0], y=EZ2[:,1], z=EZ2[:,2], mode='lines+text', text=["","z2"], textfont=dict(color="darkgreen"),name='Ez2', line=dict(color='darkgreen',width=5)))

  # Plot dos eixos x, y e z no início do elo 3
  fig.add_trace(go.Scatter3d(x=EX3[:,0], y=EX3[:,1], z=EX3[:,2], mode='lines+text', text=["","x3"], textfont=dict(color="darkblue"),name='Ex3', line=dict(color='darkblue',width=5)))
  fig.add_trace(go.Scatter3d(x=EY3[:,0], y=EY3[:,1], z=EY3[:,2], mode='lines+text', text=["","y3"], textfont=dict(color="darkred"),name='Ey3', line=dict(color='darkred',width=5)))
  fig.add_trace(go.Scatter3d(x=EZ3[:,0], y=EZ3[:,1], z=EZ3[:,2], mode='lines+text', text=["","z3"], textfont=dict(color="darkgreen"),name='Ez3', line=dict(color='darkgreen',width=5)))

  # Plot dos eixos x, y e z na início do elo 4
  fig.add_trace(go.Scatter3d(x=EXe[:,0], y=EXe[:,1], z=EXe[:,2], mode='lines+text', text=["","xe"], textfont=dict(color="darkblue"),name='Exe', line=dict(color='darkblue',width=5)))
  fig.add_trace(go.Scatter3d(x=EYe[:,0], y=EYe[:,1], z=EYe[:,2], mode='lines+text', text=["","ye"], textfont=dict(color="darkred"),name='Eye', line=dict(color='darkred',width=5)))
  fig.add_trace(go.Scatter3d(x=EZe[:,0], y=EZe[:,1], z=EZe[:,2], mode='lines+text', text=["","ze"], textfont=dict(color="darkgreen"),name='Eze', line=dict(color='darkgreen',width=5)))
  
  #Configuração de layout da imagem gerada
  fig.update_layout(
      autosize=False,
      showlegend=False,
      width=400,
      height=400,
      margin=dict(l=0,r=0,t=0,b=0)
  )

  lim = 2.5
  fig.update_scenes(
      aspectmode= "cube",
      xaxis=dict(range=[-lim,lim]),
      yaxis=dict(range=[-lim,lim]),
      zaxis=dict(range=[-lim,lim])
  )

  print("Use o mouse para interagir com a figura gerada:")
  print("Posição do efetuador final: x = [%.1f], y =[%.1f] e z =[%.1f]" %  (L3[0], L3[1], L3[2]))
  fig.show()

######################################################################################
############################ APLICANDO A CINEMÁTICA DIRETA ###########################
######################################################################################
[a1, a2, a3] = [1, 1, 1] # Tamanho dos elos
[q1, q2, q3] = [30,  -30, 30] # Ângulo das juntas

plot3jrobot(q1, a1, q2, a2, q3, a3)

Use o mouse para interagir com a figura gerada:
Posição do efetuador final: x = [1.3], y =[0.7] e z =[1.9]


# **Exercício de fixação 01:**
* Aplicando a regra da mão direita no **Exemplo 02**, analise o  isoladamente os parâmetros:
  * [q1, q2, q3] = [30, 0, 0]
  * [q1, q2, q3] = [-30, 0, 0]
  * [q1, q2, q3] = [0, 30, 0]
  * [q1, q2, q3] = [0, -30, 0]
  * [q1, q2, q3] = [0, 0, 30]
  * [q1, q2, q3] = [0, 0, -30]

# **Exercício de fixação 02:**
* Criar uma projeção para o robô SCARA;